In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import pandas as pd
import numpy as np
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder

In [2]:
iris = load_iris()
df = pd.DataFrame(iris.data, columns=iris.feature_names)
df['target'] = iris.target
df.head()

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),target
0,5.1,3.5,1.4,0.2,0
1,4.9,3.0,1.4,0.2,0
2,4.7,3.2,1.3,0.2,0
3,4.6,3.1,1.5,0.2,0
4,5.0,3.6,1.4,0.2,0


In [3]:
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42, stratify=df['target'])

In [4]:
x_train, y_train = train_df.drop(columns=['target']), train_df['target']
x_test, y_test = test_df.drop(columns=['target']), test_df['target']

In [5]:
scaler = StandardScaler()
x_train = scaler.fit_transform(x_train)
x_test = scaler.transform(x_test)

In [6]:
x_train_tensor = torch.tensor(x_train, dtype=torch.float32)
x_test_tensor = torch.tensor(x_test, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train.values, dtype=torch.long)
y_test_tensor = torch.tensor(y_test.values, dtype=torch.long)

In [7]:
class IrisClassifier(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(IrisClassifier, self).__init__()
        self.network = nn.Sequential(
            nn.Linear(input_dim, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, output_dim)
        )

    def forward(self, x):
        return self.network(x)

In [8]:
input_dim = x_train.shape[1]
hidden_dim = 16
output_dim = 3

In [9]:
model = IrisClassifier(input_dim, hidden_dim, output_dim)

In [10]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.01)

In [11]:
epochs = 500
for k in range(epochs):
  model.train()
  optimizer.zero_grad()

  predictions = model(x_train_tensor)
  loss = criterion(predictions, y_train_tensor)
  loss.backward()
  optimizer.step()

  if (k+1) % 50 == 0:
    print(f"Epoch {k+1}/{epochs}, loss: {loss.item():.4f}")


Epoch 50/500, loss: 0.0703
Epoch 100/500, loss: 0.0280
Epoch 150/500, loss: 0.0172
Epoch 200/500, loss: 0.0066
Epoch 250/500, loss: 0.0023
Epoch 300/500, loss: 0.0011
Epoch 350/500, loss: 0.0007
Epoch 400/500, loss: 0.0005
Epoch 450/500, loss: 0.0003
Epoch 500/500, loss: 0.0003


In [12]:
model.eval()
with torch.no_grad():
  y_pred = model(x_test_tensor)
  y_pred_labels = torch.argmax(y_pred, dim=1)

  accuracy = (y_pred_labels == y_test_tensor).sum().item() / len(y_test_tensor)
  print(f"Accuracy: {accuracy:.4f}")

Accuracy: 0.9667


In [17]:
def predict_iris(sepal_length, sepal_width, petal_length, petal_width):
  input_data = np.array([[sepal_length, sepal_width, petal_length, petal_width]])
  input_data - scaler.transform(input_data)
  input_tensor = torch.tensor(input_data, dtype=torch.float32)

  model.eval()
  with torch.no_grad():
    prediction = model(input_tensor)
    predicted_class = torch.argmax(prediction, dim=1).item()
  return iris.target_names[predicted_class]


In [18]:
prediction = predict_iris(5.1, 3.5, 1.4, 0.2)
print(f"Predicted Species: {prediction}")

Predicted Species: versicolor


/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
